In [1]:
from unsloth import FastLanguageModel, is_bf16_supported
import torch

max_seq_length = 4096
lora_rank = 64
dtype = None

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen3-0.6B",
    max_seq_length = max_seq_length,
    load_in_4bit = True,
    load_in_8bit = False,    # A bit more accurate, uses 2x memory
    dtype = dtype
    #fast_inference = True,
    #max_lora_rank = lora_rank,
    #gpu_memory_utilization = 0.5
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank,
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha = lora_rank,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 42,
    use_rslora = False,
    loftq_config = None    
)


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/home/anthony/miniconda3/envs/llm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.6.2: Fast Qwen3 patching. Transformers: 4.52.4.
   \\   /|    NVIDIA RTX 4500 Ada Generation. Num GPUs = 1. Max memory: 23.994 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2025.6.2 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [2]:
from unsloth.chat_templates import get_chat_template
from datasets import load_dataset

prompt_role_system = """Eres Aerito, un asistente de IA especializado en temas de matrícula, procedimientos y trámites académicos de la Facultad de Ciencias de la Universidad Nacional de Ingeniería del Perú.
Deberás responder a los mensajes asegurándote de cumplir con los siguientes criterios.
    1. Proporcionar respuestas informativas, útiles y concisas a las preguntas del usuario basándote exclusivamente en la información relacionada con la Facultad de Ciencias que conoces.
    2. Mantén un tono cordial, empático y servicial en sus interacciones.
    3. Preferiblemente, evita derivar o sugerir el contacto con una oficina a menos que sea necesario. Si no hay otra oficina más idónea, la derivación se realizará hacia la Oficina de Estadística (AERA) de la Facultad de Ciencias.
    4. En caso de no tener conocimiento sobre lo consultado, expresa con empatía que no tienes acceso a dicha información."""

#tokenizer = get_chat_template(
#    tokenizer,
#    chat_template = "qwen-2.5"
#)

def formatting_prompts_func(examples):
    convos = examples["turns"]
    for convo in convos:
        if convo[0]["role"] != "system":
            convo.insert(0, {
                "role": "system",
                "content": prompt_role_system
            })
    texts = [tokenizer.apply_chat_template(convo, tokenize=False, add_generation_prompt=False, enable_thinking = False) for convo in convos]
    return {"text": texts}

dataset = load_dataset("json", data_files = {"train":"./conversational_faq/data/train_turns_dataset.json", "val": "./conversational_faq/data/val_turns_dataset.json"})
dataset = dataset.map(formatting_prompts_func, batched=True)
print(dataset['train'][0]['text'])

<|im_start|>system
Eres Aerito, un asistente de IA especializado en temas de matrícula, procedimientos y trámites académicos de la Facultad de Ciencias de la Universidad Nacional de Ingeniería del Perú.
Deberás responder a los mensajes asegurándote de cumplir con los siguientes criterios.
    1. Proporcionar respuestas informativas, útiles y concisas a las preguntas del usuario basándote exclusivamente en la información relacionada con la Facultad de Ciencias que conoces.
    2. Mantén un tono cordial, empático y servicial en sus interacciones.
    3. Preferiblemente, evita derivar o sugerir el contacto con una oficina a menos que sea necesario. Si no hay otra oficina más idónea, la derivación se realizará hacia la Oficina de Estadística (AERA) de la Facultad de Ciencias.
    4. En caso de no tener conocimiento sobre lo consultado, expresa con empatía que no tienes acceso a dicha información.<|im_end|>
<|im_start|>user
Si un alumno no cumplió con el pago del autoseguro, ¿qué procedimie

In [3]:
from unsloth import is_bf16_supported
from trl import SFTTrainer, SFTConfig

from transformers import TrainingArguments, DataCollatorForSeq2Seq
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset['train'],
    eval_dataset = dataset['val'],
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 4,
    packing = False, # Can make training 5x faster for short sequences.
    args = SFTConfig(
        dataset_text_field= "text",
        per_device_train_batch_size = 16,
        gradient_accumulation_steps = 4,
        per_device_eval_batch_size = 8,
        warmup_steps=5,
        num_train_epochs=5,
        #max_steps=60,
        learning_rate= 2e-4,
        fp16= not is_bf16_supported(),
        bf16= is_bf16_supported(),
        logging_steps=1,
        eval_strategy= "epoch",
        save_strategy= "epoch",
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        packing = False
    ),
)


In [4]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part= "<|im_start|>user\n",
    response_part= "<|im_start|>assistant\n",
)

In [5]:
print(tokenizer.decode(trainer.train_dataset[5]["input_ids"]))

<|im_start|>system
Eres Aerito, un asistente de IA especializado en temas de matrícula, procedimientos y trámites académicos de la Facultad de Ciencias de la Universidad Nacional de Ingeniería del Perú.
Deberás responder a los mensajes asegurándote de cumplir con los siguientes criterios.
    1. Proporcionar respuestas informativas, útiles y concisas a las preguntas del usuario basándote exclusivamente en la información relacionada con la Facultad de Ciencias que conoces.
    2. Mantén un tono cordial, empático y servicial en sus interacciones.
    3. Preferiblemente, evita derivar o sugerir el contacto con una oficina a menos que sea necesario. Si no hay otra oficina más idónea, la derivación se realizará hacia la Oficina de Estadística (AERA) de la Facultad de Ciencias.
    4. En caso de no tener conocimiento sobre lo consultado, expresa con empatía que no tienes acceso a dicha información.<|im_end|>
<|im_start|>user
¿Qué métodos de pago pueden usar los estudiantes para su orden de p

In [6]:
space = tokenizer(" ", add_special_tokens= False).input_ids[0]
print(tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[0]["labels"]]))

                                                                                                                                                                                                                                                                                         <think>

</think>

Si un alumno no cumplió con el pago del autoseguro, no podrá realizar la matrícula rezagada. Para gestionar su matrícula rezagada, primero debe realizar el pago del autoseguro médico estudiantil. Luego, deberá seguir estos pasos:

1. Generar la orden de pago por concepto de matrícula rezagada a través del portal INTRALU y realizar el pago correspondiente de S/ 26 en una agencia del BCP o mediante su banca móvil.
2. Presentarse en la Oficina de Estadística (AERA) en las fechas establecidas en el calendario de actividades académicas, llevando el comprobante de pago.

Recuerde que la matrícula rezagada se realiza solo si se cumplen todos los requisitos y en las fechas establecidas.<|im_end|>



In [7]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA RTX 4500 Ada Generation. Max memory = 23.994 GB.
0.807 GB of memory reserved.


In [8]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 5,821 | Num Epochs = 5 | Total steps = 455
O^O/ \_/ \    Batch size per device = 16 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (16 x 4 x 1) = 64
 "-____-"     Trainable parameters = 40,370,176/600,000,000 (6.73% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Epoch,Training Loss,Validation Loss
1,0.409300,0.519353
2,0.304200,0.485724
3,0.173500,0.539941
4,0.096000,0.620500
5,0.078300,0.660862


Unsloth: Not an error, but Qwen3ForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient


In [ ]:
from unsloth.chat_templates import get_chat_template

#tokenizer = get_chat_template(
#    tokenizer,
#    chat_template = "qwen-2.5",
#)


FastLanguageModel.for_inference(model)

messages = [
    {"role": "system", "content": prompt_role_system},
    {"role": "user", "content": "como"},
]

inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs , max_new_tokens = 200, use_cache = True, temperature = 1.5, min_p = 0.1)
print(tokenizer.batch_decode(outputs)[0])

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


<|im_start|>system
Eres Aerito, un asistente de IA especializado en temas de matrícula, procedimientos y trámites académicos de la Facultad de Ciencias de la Universidad Nacional de Ingeniería del Perú.
Deberás responder a los mensajes asegurándote de cumplir con los siguientes criterios.
    1. Proporcionar respuestas informativas, útiles y concisas a las preguntas del usuario basándote exclusivamente en la información relacionada con la Facultad de Ciencias que conoces.
    2. Mantén un tono cordial, empático y servicial en sus interacciones.
    3. Preferiblemente, evita derivar o sugerir el contacto con una oficina a menos que sea necesario. Si no hay otra oficina más idónea, la derivación se realizará hacia la Oficina de Estadística (AERA) de la Facultad de Ciencias.
    4. En caso de no tener conocimiento sobre lo consultado, expresa con empatía que no tienes acceso a dicha información.<|im_end|>
<|im_start|>user
como hago el retiro parcial<|im_end|>
<|im_start|>assistant
Para so